In [ ]:
## Using CVXPY to solve a multi-scenario optimization problem
## Dual-Optimization setup to minimize residual load realtime

import numpy as np
import pandas as pd
import cvxpy as cp

linecut = 96
scenario1 = pd.read_csv('data/Scenario1.csv').iloc[:linecut]
scenario2 = pd.read_csv('data/Scenario2.csv').iloc[:linecut]
scenario3 = pd.read_csv('data/Scenario3.csv').iloc[:linecut]
scenario4 = pd.read_csv('data/Scenario4.csv').iloc[:linecut]
scenario5 = pd.read_csv('data/Scenario5.csv').iloc[:linecut]
scenarios_data = [scenario1, scenario2, scenario3, scenario4, scenario5]

# Fixed cost parameters
wind_cost_per_mw = 250
base_load_cost_per_mw = 360
peak_load_cost_per_mw = 200
load_following_option_fee = 50
load_following_exercise_fee = 18

max_wind_capacity = 20
max_generator_capacity = 10
solar_capacity = 1

wind_capacity = cp.Variable()
baseload_capacity = cp.Variable() # [MW]
peak_capacity = cp.Variable() # [MW]
load_following_capacity = cp.Variable()
x = [wind_capacity, baseload_capacity, peak_capacity, load_following_capacity]

constraints = [
    wind_capacity >= 0,
    baseload_capacity >= 0,
    peak_capacity >= 0,
    load_following_capacity >= 0,
    wind_capacity <= max_wind_capacity,
    baseload_capacity + peak_capacity + load_following_capacity <= max_generator_capacity
]

# Scenario cost with real-time load-following optimization
def scenario_cost(x, demand, real_time_price, wind_factor, solar_factor):

    # Fixed cost regardless of realtime events
    fixed_cost = (x[0] * wind_cost_per_mw +
                x[1] * base_load_cost_per_mw +
                x[2] * peak_load_cost_per_mw +
                x[3] * load_following_option_fee)

    demand_mw = demand.values / 1000
    real_time_price = real_time_price.values
    solar_gen = solar_capacity * solar_factor.values
    wind_gen = x[0] * wind_factor.values
    base_gen = x[1]
    peak_gen = x[2]

    # Initialize real-time cost
    real_time_cost = 0

    # LOAD FOLLOWING OPTIMIZATION
    for t in range(len(demand_mw)):
        
        # Unmet demand at time t (index in 15min interval)
        if 8 <= float(t)*4/60 <= 16:
            unmet_demand_t = demand_mw[t] - (solar_gen[t] + wind_gen[t] + base_gen + peak_gen)
        else:
            unmet_demand_t = demand_mw[t] - (solar_gen[t] + wind_gen[t] + base_gen)
        print(unmet_demand_t)
        
        # Load-following generation variable for period t
        load_follow_gen_t = cp.Variable()
        spot_load_t = cp.Variable()
        
        # Real-time optimization for load following cost at time t
        real_time_cost_t = load_follow_gen_t * (real_time_price[t] + load_following_exercise_fee) + spot_load_t * real_time_price[t]
        
        # Constraints for load following optimization
        load_follow_constraints = [
            load_follow_gen_t >= 0,                             # Load following is non-negative
            load_follow_gen_t <= x[3],                          # Load following capacity constraint
            spot_load_t >= 0,                                   # Spot load is non-negative
            load_follow_gen_t + spot_load_t >= unmet_demand_t   # Ensure load following covers unmet demand
        ]
        # Unmet demand is critical component of the optimization. Set here but not on upper level
        
        # Solve the sub-problem to minimize real-time cost at time t
        real_time_problem = cp.Problem(cp.Minimize(real_time_cost_t), load_follow_constraints)
        real_time_problem.solve(solver=cp.GUROBI)
        
        # Accumulate real-time costs
        real_time_cost += real_time_cost_t.value
    print(real_time_problem.value)
    # print(fixed_cost)

    # Total scenario cost
    return fixed_cost + real_time_cost

# Aggregate cost function across all scenarios
def aggregate_cost(x):
    total_cost = 0
    for scenario in scenarios_data:
        total_cost += scenario_cost(
            x,
            demand=scenario['Demand [kW]'],
            real_time_price=scenario['Real Time Price [$/MWh]'],
            wind_factor=scenario['Wind Power Factor [p.u.]'],
            solar_factor=scenario['Solar Power Factor [p.u.]']
        )
    return total_cost / len(scenarios_data)

# Objective: Minimize the average cost across all scenarios
objective = cp.Minimize(aggregate_cost(x))

# Define and solve the problem using the Gurobi solver
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)

# Output results
if problem.status == cp.OPTIMAL:
    output = {
        "Minimum expected cost": problem.value,
        "Wind capacity (MW)": wind_capacity.value,
        "Baseload capacity (MW)": baseload_capacity.value,
        "Peak load capacity (MW)": peak_capacity.value,
        "Load following capacity (MW)": load_following_capacity.value
    }
else:
    output = {"error": problem.status}

output


7.37568 + -Promote(var216918, (96,)) @ [0.05 0.05 0.05 0.04 0.04 0.05 0.06 0.06 0.05 0.01 0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.04 0.   0.01 0.04
 0.05 0.07 0.09 0.11 0.12 0.13 0.15 0.16 0.15 0.14 0.13 0.13 0.15 0.16
 0.15 0.15 0.15 0.13 0.13 0.13 0.13 0.14 0.13 0.08 0.16 0.13][0] + var216919
7.31808 + -Promote(var216918, (96,)) @ [0.05 0.05 0.05 0.04 0.04 0.05 0.06 0.06 0.05 0.01 0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.04 0.   0.01 0.04
 0.05 0.07 0.09 0.11 0.12 0.13 0.15 0.16 0.15 0.14 0.13 0.13 0.15 0.16
 0.15 0.1

{'Minimum expected cost': 0.0,
 'Wind capacity (MW)': array(0.),
 'Baseload capacity (MW)': array(0.),
 'Peak load capacity (MW)': array(0.),
 'Load following capacity (MW)': array(0.)}

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp

# Load data for all scenarios
scenario1 = pd.read_csv('data/Scenario1.csv')
scenario2 = pd.read_csv('data/Scenario2.csv')
scenario3 = pd.read_csv('data/Scenario3.csv')
scenario4 = pd.read_csv('data/Scenario4.csv')
scenario5 = pd.read_csv('data/Scenario5.csv')
linecut = 96
scenario1 = scenario1.iloc[:linecut]
scenario2 = scenario2.iloc[:linecut]
scenario3 = scenario3.iloc[:linecut]
scenario4 = scenario4.iloc[:linecut]
scenario5 = scenario5.iloc[:linecut]
scenarios_data = [scenario1, scenario2, scenario3, scenario4, scenario5]

# Fixed contract cost parameters
wind_cost_per_mw = 250
base_load_cost_per_mwh = 360
peak_load_cost_per_mwh = 200
load_following_option_fee = 50
load_following_exercise_fee = 18

max_wind_capacity = 20
max_generator_capacity = 10
solar_capacity = 1

# Define cvxpy variables for contract capacities
wind_capacity = cp.Variable()
baseload_capacity = cp.Variable()
peak_capacity = cp.Variable()
load_following_capacity = cp.Variable()
x = [wind_capacity, baseload_capacity, peak_capacity, load_following_capacity]

# Constraints for capacities
constraints = [
    wind_capacity >= 0,
    baseload_capacity >= 0,
    peak_capacity >= 0,
    load_following_capacity >= 0,
    wind_capacity <= max_wind_capacity,
    baseload_capacity + peak_capacity + load_following_capacity <= max_generator_capacity
]

# Define vectorized aggregate cost function
def scenario_cost(x, demand, real_time_price, wind_factor, solar_factor):  
    # Define fixed costs
    fixed_cost = (x[0] * wind_cost_per_mw +
                  x[1] * base_load_cost_per_mwh +
                  x[2] * peak_load_cost_per_mwh +
                  x[3] * load_following_option_fee)

    demand_mw = demand.values / 1000
    real_time_price = real_time_price.values
    solar_gen = solar_capacity * solar_factor.values
    wind_gen = x[0] * wind_factor.values
    base_gen = x[1]
    peak_gen = x[2]

    # Calculate unmet demand and cost for load following generation
    unmet_demand = demand_mw - (solar_gen + wind_gen + base_gen + peak_gen)
    load_follow_gen = cp.maximum(0, unmet_demand)
    real_time_cost = cp.sum(load_follow_gen @ (real_time_price + load_following_exercise_fee))
    ##### CHECK OUT ACCURACY
    return fixed_cost + real_time_cost

# Aggregate (avg) cost function across all scenarios
def aggregate_cost(x):
    total_cost = 0
    for scenario in scenarios_data:
        total_cost += scenario_cost(
            x,
            demand=scenario['Demand [kW]'],
            real_time_price=scenario['Real Time Price [$/MWh]'],
            wind_factor=scenario['Wind Power Factor [p.u.]'],
            solar_factor=scenario['Solar Power Factor [p.u.]']
        )
    return total_cost / len(scenarios_data)

# Objective: Minimize the average cost across all scenarios
objective = cp.Minimize(aggregate_cost(x))

# Define and solve the problem using Gurobi solver
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)

# Output results
if problem.status == cp.OPTIMAL:
    output = {
        "Minimum expected cost": problem.value,
        "Wind capacity (MW)": wind_capacity.value,
        "Baseload capacity (MW)": baseload_capacity.value,
        "Peak load capacity (MW)": peak_capacity.value,
        "Load following capacity (MW)": load_following_capacity.value
    }
else:
    output = {"error": problem.status}

output